In [101]:
from pyspark.sql import SparkSession
import json
import pandas as pd

In [102]:
spark = spark = SparkSession.builder \
    .appName("pyspark_test") \
    .getOrCreate()

In [103]:
file_path = '../data/Team_Liquid_vs_Sentinels.json'
with open(file_path, 'r') as f:
    data = json.load(f)

### get match event data

In [104]:
match_name = list(data.keys())[0]
print(match_name)

team1, team2 = match_name.split('_vs_')
team1_name = team1.replace('_', ' ')
team2_name = team2.replace('_', ' ')


event_stats = data[match_name][0]
event_stats

event_stats_df_pd = pd.DataFrame(data[list(data.keys())[0]][0], index=[0])

event_stats_df = spark.createDataFrame(event_stats_df_pd)
event_stats_df.show()



Team_Liquid_vs_Sentinels
+--------------------+----------+-----------+-------------+
|               Event|      Date|      Stage|        Round|
+--------------------+----------+-----------+-------------+
|Champions Tour 20...|2025-02-23|Swiss Stage|Round 2 (0-1)|
+--------------------+----------+-----------+-------------+



### get team map data

In [105]:
map_list = list(data[match_name][1].keys())

map_scored_df_pd = pd.DataFrame(columns=['Map', 'Winner', 'TeamOneName', 'TeamTwoName', 'TeamOneFinalScore', 'TeamOneCTScore', 'TeamOneTScore', 'TeamOneOTScore', 'TeamTwoFinalScore', 'TeamTwoCTScore', 'TeamTwoTScore', 'TeamTwoOTScore'])

for i in map_list:
    map_name = i

    team1_stats = data[match_name][1][i]['Team Stats: '][team1_name]
    team2_stats = data[match_name][1][i]['Team Stats: '][team2_name]

    team1_score = int(team1_stats['final'])
    team2_score = int(team2_stats['final'])
    

    winner = team1_name if int(team1_score) > int(team2_score) else team2_name

    temp_df1 = pd.DataFrame({'Map': map_name, 'Winner': winner, 'TeamOneName': team1_name, 'TeamTwoName': team2_name, 'TeamOneFinalScore': team1_stats['final'], 'TeamOneCTScore': team1_stats['CT'], 'TeamOneTScore' : team1_stats['T'], 'TeamOneOTScore' : team1_stats['OT'], 'TeamTwoFinalScore': team2_stats['final'], 'TeamTwoCTScore' : team2_stats['CT'], 'TeamTwoTScore' : team2_stats['T'], 'TeamTwoOTScore' : team2_stats['CT']}, index=[0])

    map_scored_df_pd = pd.concat([map_scored_df_pd, temp_df1], ignore_index=True)


map_scored_df = spark.createDataFrame(map_scored_df_pd)
map_scored_df.show()

+-----+-----------+-----------+-----------+-----------------+--------------+-------------+--------------+-----------------+--------------+-------------+--------------+
|  Map|     Winner|TeamOneName|TeamTwoName|TeamOneFinalScore|TeamOneCTScore|TeamOneTScore|TeamOneOTScore|TeamTwoFinalScore|TeamTwoCTScore|TeamTwoTScore|TeamTwoOTScore|
+-----+-----------+-----------+-----------+-----------------+--------------+-------------+--------------+-----------------+--------------+-------------+--------------+
|Lotus|  Sentinels|Team Liquid|  Sentinels|                7|             2|            5|             0|               13|             7|            6|             7|
|Abyss|Team Liquid|Team Liquid|  Sentinels|               16|             7|            5|             4|               14|             7|            5|             7|
| Bind|Team Liquid|Team Liquid|  Sentinels|               13|            11|            2|             0|                1|             0|            1|        

### get map player data

In [106]:
map_list = list(data[match_name][1].keys())

map_player_stats_df_pd = pd.DataFrame(columns=['Map', 'PlayerName', 'Agent', 'Team', 'ACS_Overall','ACS_T', 'ACS_CT', 'Kills_Overall', 'Kills_T', 'Kills_CT',  'Deaths_Overall', 'Deaths_T', 'Deaths_CT', 'Assists_Overall', 'Assists_T', 'Assists_CT', 'KAST_Overall', 'KAST_T', 'KAST_CT', 'ADR_Overall', 'ADR_T', 'ADR_CT', 'HSPercentage_Overall', 'HSPercentage_T', 'HSPercentage_CT', 'FirstKills_Overall', 'FirstKills_T', 'FirstKills_CT', 'FirstDeaths_Overall', 'FirstDeaths_T', 'FirstDeaths_CT'])

for i in map_list:
    map_name = i

    map_player_stats = data[match_name][1][i]['Player Stats']
    
    for player in map_player_stats:
        player_name = player
        player_stats = map_player_stats[player]

        temp_df3 = pd.DataFrame({'Map' : map_name,'PlayerName': player_name, 'Agent': player_stats['Agent'], 'Team' : player_stats['Team'], 'ACS_Overall': player_stats['ACS']['All'], 'ACS_T':player_stats['ACS']['T'], 'ACS_CT': player_stats['ACS']['CT'], 'Kills_Overall' : player_stats['Elims']['All'], 'Kills_T': player_stats['Elims']['T'], 'Kills_CT': player_stats['Elims']['CT'], 'Deaths_Overall' : player_stats['Deaths']['All'], 'Deaths_T': player_stats['Deaths']['T'], 'Deaths_CT': player_stats['Deaths']['CT'], 'Assists_Overall' : player_stats['Assists']['All'], 'Assists_T': player_stats['Assists']['T'], 'Assists_CT': player_stats['Assists']['CT'], 'KAST_Overall' :player_stats['KAST']['All'], 'KAST_T': player_stats['KAST']['T'], 'KAST_CT': player_stats['KAST']['CT'], 'ADR_Overall' : player_stats['ADR']['All'], 'ADR_T': player_stats['ADR']['T'], 'ADR_CT': player_stats['ADR']['CT'], 'HSPercentage_Overall' : player_stats['HS_percentage']['All'], 'HSPercentage_T': player_stats['HS_percentage']['T'], 'HSPercentage_CT': player_stats['HS_percentage']['CT'], 'FirstKills_Overall' : player_stats['First Kills']['All'], 'FirstKills_T': player_stats['First Kills']['T'], 'FirstKills_CT': player_stats['First Kills']['CT'], 'FirstDeaths_Overall' : player_stats['First Deaths']['All'], 'FirstDeaths_T': player_stats['First Deaths']['T'], 'FirstDeaths_CT': player_stats['First Deaths']['CT']}, index=[0])

        map_player_stats_df_pd = pd.concat([map_player_stats_df_pd, temp_df3], ignore_index=True)

map_player_stats_df = spark.createDataFrame(map_player_stats_df_pd)
map_player_stats_df.show(5)

+-----+----------+------+----+-----------+-----+------+-------------+-------+--------+--------------+--------+---------+---------------+---------+----------+------------+------+-------+-----------+-----+------+--------------------+--------------+---------------+------------------+------------+-------------+-------------------+-------------+--------------+
|  Map|PlayerName| Agent|Team|ACS_Overall|ACS_T|ACS_CT|Kills_Overall|Kills_T|Kills_CT|Deaths_Overall|Deaths_T|Deaths_CT|Assists_Overall|Assists_T|Assists_CT|KAST_Overall|KAST_T|KAST_CT|ADR_Overall|ADR_T|ADR_CT|HSPercentage_Overall|HSPercentage_T|HSPercentage_CT|FirstKills_Overall|FirstKills_T|FirstKills_CT|FirstDeaths_Overall|FirstDeaths_T|FirstDeaths_CT|
+-----+----------+------+----+-----------+-----+------+-------------+-------+--------+--------------+--------+---------+---------------+---------+----------+------------+------+-------+-----------+-----+------+--------------------+--------------+---------------+------------------+---

### get match aggregate data

### team data

In [107]:
map_scored_df_pd = map_scored_df_pd.astype({ 'TeamOneFinalScore' : int, 'TeamOneCTScore' : int, 'TeamOneTScore' : int, 'TeamOneOTScore' : int, 'TeamTwoFinalScore' : int, 'TeamTwoCTScore' : int, 'TeamTwoTScore' : int, 'TeamTwoOTScore' : int})
map_scored_df_pd

,Map,Winner,TeamOneName,TeamTwoName,TeamOneFinalScore,TeamOneCTScore,TeamOneTScore,TeamOneOTScore,TeamTwoFinalScore,TeamTwoCTScore,TeamTwoTScore,TeamTwoOTScore
0,Lotus,Sentinels,Team Liquid,Sentinels,7,2,5,0,13,7,6,7
1,Abyss,Team Liquid,Team Liquid,Sentinels,16,7,5,4,14,7,5,7
2,Bind,Team Liquid,Team Liquid,Sentinels,13,11,2,0,1,0,1,0


In [108]:
column_sums = map_scored_df_pd.sum(axis=0)

match_scored_df_pd_sums = column_sums.to_frame()
match_scored_df_pd_t = match_scored_df_pd_sums.transpose()
match_scored_df_pd = match_scored_df_pd_t[['TeamOneName','TeamTwoName','TeamOneFinalScore', 'TeamOneCTScore', 'TeamOneTScore', 'TeamOneOTScore', 'TeamTwoFinalScore', 'TeamTwoCTScore', 'TeamTwoTScore', 'TeamTwoOTScore']]
match_scored_df_pd.at[0, 'TeamOneName'] = team1_name
match_scored_df_pd.at[0, 'TeamTwoName'] = team2_name
match_scored_df_pd

,TeamOneName,TeamTwoName,TeamOneFinalScore,TeamOneCTScore,TeamOneTScore,TeamOneOTScore,TeamTwoFinalScore,TeamTwoCTScore,TeamTwoTScore,TeamTwoOTScore
0,Team Liquid,Sentinels,36,20,12,4,28,14,12,14


### Player Data

In [109]:
map_player_stats_df_pd_trim = map_player_stats_df_pd.copy()

map_player_stats_df_pd_trim['KAST_Overall'] = map_player_stats_df_pd_trim['KAST_Overall'].str.replace('%', '').astype(float)
map_player_stats_df_pd_trim['KAST_CT'] = map_player_stats_df_pd_trim['KAST_CT'].str.replace('%', '').astype(float)
map_player_stats_df_pd_trim['KAST_T'] = map_player_stats_df_pd_trim['KAST_T'].str.replace('%', '').astype(float)
map_player_stats_df_pd_trim['HSPercentage_Overall'] = map_player_stats_df_pd_trim['HSPercentage_Overall'].str.replace('%', '').astype(float)
map_player_stats_df_pd_trim['HSPercentage_CT'] = map_player_stats_df_pd_trim['HSPercentage_CT'].str.replace('%', '').astype(float)
map_player_stats_df_pd_trim['HSPercentage_T'] = map_player_stats_df_pd_trim['HSPercentage_T'].str.replace('%', '').astype(float)

map_player_stats_df_pd_trim = map_player_stats_df_pd_trim.astype({'ACS_Overall': 'int', 'ACS_T': 'int', 'ACS_CT':  'int', 'Kills_Overall' : 'int', 'Kills_T': 'int', 'Kills_CT': 'int', 'Deaths_Overall' : 'int', 'Deaths_T': 'int', 'Deaths_CT': 'int', 'Assists_Overall' : 'int', 'Assists_T': 'int', 'Assists_CT': 'int', 'KAST_Overall' : 'float', 'KAST_T':  'float', 'KAST_CT':  'float', 'ADR_Overall' :  'int', 'ADR_T':  'int', 'ADR_CT':  'int', 'HSPercentage_Overall' :  'float', 'HSPercentage_T':  'float', 'HSPercentage_CT':  'float', 'FirstKills_Overall' : 'int', 'FirstKills_T': 'int', 'FirstKills_CT': 'int', 'FirstDeaths_Overall' : 'int', 'FirstDeaths_T': 'int', 'FirstDeaths_CT': 'int'})

map_player_stats_df_pd_trim

,Map,PlayerName,Agent,Team,ACS_Overall,ACS_T,ACS_CT,Kills_Overall,Kills_T,Kills_CT,...,ADR_CT,HSPercentage_Overall,HSPercentage_T,HSPercentage_CT,FirstKills_Overall,FirstKills_T,FirstKills_CT,FirstDeaths_Overall,FirstDeaths_T,FirstDeaths_CT
0,Lotus,nAts,Cypher,TL,293,254,354,22,12,10,...,248,39.0,44.0,33.0,1,0,1,1,1,0
1,Lotus,Keiko,Omen,TL,192,276,68,14,12,2,...,51,59.0,58.0,67.0,3,3,0,4,2,2
2,Lotus,paTiTek,Breach,TL,151,157,143,11,7,4,...,88,21.0,28.0,13.0,0,0,0,3,0,3
3,Lotus,kamyk,Tejo,TL,136,112,173,9,4,5,...,88,42.0,17.0,86.0,2,1,1,1,0,1
4,Lotus,kamo,Yoru,TL,97,112,76,6,4,2,...,49,17.0,13.0,33.0,1,1,0,4,4,0
5,Lotus,bang,Omen,SEN,231,240,225,17,8,9,...,148,31.0,40.0,27.0,2,1,1,1,0,1
6,Lotus,zekken,Raze,SEN,255,364,184,18,10,8,...,135,30.0,41.0,23.0,4,3,1,3,1,2
7,Lotus,N4RRATE,Fade,SEN,239,258,227,17,7,10,...,141,27.0,15.0,44.0,4,0,4,1,0,1
8,Lotus,johnqt,Viper,SEN,200,171,220,15,6,9,...,128,17.0,23.0,15.0,1,0,1,1,0,1
9,Lotus,Zellsis,Vyse,SEN,110,92,123,8,2,6,...,80,35.0,33.0,35.0,2,2,0,1,1,0


In [110]:
test_df = map_player_stats_df_pd_trim.groupby('PlayerName').agg({'ACS_Overall': 'mean', 'ACS_T': 'mean', 'ACS_CT':  'mean', 'Kills_Overall' : 'sum', 'Kills_T': 'sum', 'Kills_CT': 'sum', 'Deaths_Overall' : 'sum', 'Deaths_T': 'sum', 'Deaths_CT': 'sum', 'Assists_Overall' : 'sum', 'Assists_T': 'sum', 'Assists_CT': 'sum', 'KAST_Overall' : 'mean', 'KAST_T':  'mean', 'KAST_CT':  'mean', 'ADR_Overall' :  'mean', 'ADR_T':  'mean', 'ADR_CT':  'mean', 'HSPercentage_Overall' :  'mean', 'HSPercentage_T':  'mean', 'HSPercentage_CT':  'mean', 'FirstKills_Overall' : 'sum', 'FirstKills_T': 'sum', 'FirstKills_CT': 'sum', 'FirstDeaths_Overall' : 'sum', 'FirstDeaths_T': 'sum', 'FirstDeaths_CT': 'sum'}).round(2)
test_df

,ACS_Overall,ACS_T,ACS_CT,Kills_Overall,Kills_T,Kills_CT,Deaths_Overall,Deaths_T,Deaths_CT,Assists_Overall,...,ADR_CT,HSPercentage_Overall,HSPercentage_T,HSPercentage_CT,FirstKills_Overall,FirstKills_T,FirstKills_CT,FirstDeaths_Overall,FirstDeaths_T,FirstDeaths_CT
PlayerName,,,,,,,,,,,,,,,,,,,,,
Keiko,212.33,335.00,167.33,54,28,26,9,20,20,27,...,109.00,37.67,41.33,39.00,7,5,2,6,2,4
N4RRATE,230.00,220.33,294.33,55,25,30,4,28,21,21,...,174.67,25.67,25.00,28.33,9,2,7,3,2,1
Zellsis,174.67,164.67,153.67,34,15,19,4,32,20,29,...,97.00,24.67,22.00,36.00,4,3,1,5,3,2
bang,186.33,188.33,154.00,38,22,16,4,24,21,20,...,94.67,29.67,34.00,51.67,7,4,3,6,2,4
johnqt,160.00,159.33,149.00,37,22,15,4,26,19,18,...,105.67,20.67,25.00,13.00,1,0,1,1,0,1
kamo,190.33,149.67,206.00,41,13,28,9,23,23,12,...,132.33,25.67,28.33,29.33,11,4,7,14,10,4
kamyk,177.00,144.00,186.67,39,15,24,12,21,23,13,...,116.33,30.67,48.67,43.67,4,1,3,2,0,2
nAts,305.33,211.00,359.00,66,20,46,10,20,20,13,...,231.33,27.67,45.00,23.67,6,0,6,6,4,2
paTiTek,151.67,197.00,131.00,32,18,14,11,21,22,36,...,85.33,24.33,29.00,19.00,4,3,1,4,0,4
